In [1]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from itertools import product
from lightgbm import LGBMRegressor
import xgboost as xgb
from xgboost import XGBRegressor
def MSE(series_true, series_predicted):
    return sum((series_true - series_predicted)**2 / len(series_true))**0.5

Here are the results of modelling with data that contains 4 features: date_block_num, shop_id, item_id and item_category_id

xgboost

In [22]:
xgb_model = XGBRegressor(n_estimators=100, max_depth=6)

In [24]:
xgb_model.fit(train_X, train_y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='approx', validate_parameters=1, verbosity=None)

In [27]:
MSE(test_y, xgb_model.predict(test_X))

4.995693356637923

xgboost with clipping

In [18]:
xgb_model = XGBRegressor(n_estimators=100, max_depth=6)
xgb_model.fit(train_X, train_y.clip(0, 20))

NameError: name 'MSE' is not defined

In [23]:
MSE(test_y.clip(0, 20), xgb_model.predict(test_X))

1.0138673005794732

In [24]:
MSE(test_y, xgb_model.predict(test_X))

5.2544421849370195

boosting

In [85]:
boosting = GradientBoostingRegressor(n_estimators=200)

In [86]:
boosting.fit(train_X, train_y)

GradientBoostingRegressor(n_estimators=200)

In [87]:
predicted = boosting.predict(test_X)

In [33]:
tree = DecisionTreeRegressor(max_depth=20)

In [34]:
tree.fit(all_sales.drop(columns=['item_cnt_day']), all_sales.item_cnt_day.clip(0, 20))

DecisionTreeRegressor(max_depth=20)

In [35]:
test = pd.read_csv('data/test.csv', index_col='ID')
test['date_block_num'] = 34
test = test.merge(items.drop(columns=['item_name']), on='item_id', how='left')
test = test[['date_block_num', 'shop_id', 'item_id', 'item_category_id']]

submission = pd.read_csv('data/sample_submission.csv')
predicted = tree.predict(test)
submission.item_cnt_month = predicted
submission.to_csv('data/my_submission_3.csv', index=None)

In [36]:
test

,date_block_num,shop_id,item_id,item_category_id
0,34,5,5037,19
1,34,5,5320,55
2,34,5,5233,19
3,34,5,5232,23
4,34,5,5268,20
...,...,...,...,...
214195,34,45,18454,55
214196,34,45,16188,64
214197,34,45,15757,55
214198,34,45,19648,40


tree score = 1.15164

In [40]:
xgb_model = XGBRegressor(n_estimators=100, max_depth=6)
xgb_model.fit(all_sales.drop(columns=['item_cnt_day']), all_sales.item_cnt_day.clip(0, 20))

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='approx', validate_parameters=1, verbosity=None)

In [41]:
xgb_predicted = xgb_model.predict(test)
submission.item_cnt_month = xgb_predicted
submission.to_csv('data/my_submission_4.csv', index=None)

xgb score = 1.09442

Here is feature engineering.

The function that creates lag features

In [2]:
def lag_feature(df, lag, col, merge_cols):        
    temp = df[merge_cols + [col]]
    temp = temp.groupby(merge_cols).agg({f'{col}':'first'}).reset_index()
    temp.columns = merge_cols + [f'{col}_lag{lag}']
    temp['date_block_num'] += lag
    df = pd.merge(df, temp, on=merge_cols, how='left')
    df[f'{col}_lag{lag}'] = df[f'{col}_lag{lag}'].fillna(0).astype('float32')
    return df

Expanding the test table for concatenating it with the train table

In [3]:
test = pd.read_csv('data/test.csv', index_col='ID')
test['date_block_num'] = 34
test = test[['date_block_num', 'shop_id', 'item_id']]
test['item_cnt_day'] = test['item_price'] = 0

Here the table train + test is expanded by adding rows for each month such that all rows of each month contain info about all (shop, item) elements if at least one of them is "active" (there are rows with this value).

In [4]:
train_sales = pd.read_csv('data/sales_train.csv').drop(columns='date')
train_sales = pd.concat([train_sales, test], ignore_index=True)
train_sales = train_sales.groupby(['shop_id', 'item_id', 'date_block_num']).agg({'item_cnt_day': 'sum'}).reset_index()

train = []

for i in train_sales['date_block_num'].unique():
    all_shop = train_sales.loc[train_sales['date_block_num']==i, 'shop_id'].unique()
    all_item = train_sales.loc[train_sales['date_block_num']==i, 'item_id'].unique()
    train.append(np.array(list(product([i], all_shop, all_item))))

idx_features = ['date_block_num', 'shop_id', 'item_id']
train = pd.DataFrame(np.vstack(train), columns=idx_features)

all_sales = train.merge(train_sales, how='left', on=idx_features)
all_sales.fillna(value=0, inplace=True)

Reducing the memory size.

In [5]:
del train_sales
del train
del all_shop
del all_item

In [6]:
all_sales = all_sales.astype({
    'date_block_num': 'int8',
    'shop_id': 'int32',
    'item_id': 'int32',
    'item_cnt_day': 'int32'
})

In [7]:
all_sales.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11128050 entries, 0 to 11128049
Data columns (total 4 columns):
 #   Column          Non-Null Count     Dtype
---  ------          --------------     -----
 0   date_block_num  11128050 non-null  int8 
 1   shop_id         11128050 non-null  int32
 2   item_id         11128050 non-null  int32
 3   item_cnt_day    11128050 non-null  int32
dtypes: int32(3), int8(1)
memory usage: 222.9 MB


Replacing all shop_ids that equals to 0, 1 or 10 to the value 57, 58 or 11 respectively.

In [8]:
all_sales.shop_id.replace(to_replace=[0, 1, 10], value=[57, 58, 11], inplace=True)

In [9]:
item_categories = pd.read_csv('data/item_categories_1.csv', dtype='int8')
shops = pd.read_csv('data/shops_1.csv', dtype='int8')

items = pd.read_csv('data/items.csv')
items.drop(columns='item_name', inplace=True)
items = items.astype('int32')

As I used one-hot-encoding for encoding the items and shops, now I use label encoding. This function "converts" one-hot-encoding to label encoding.

In [10]:
def from_one_hot_to_label(df, cols, dtype='int8'):
    return (df[cols] * range(len(cols))).sum(axis=1).astype(dtype)

In [11]:
item_super_categories = item_categories.columns[item_categories.columns.str.endswith('_super_category')]
item_categories['super_category_label'] = from_one_hot_to_label(item_categories, item_super_categories)

item_simple_categories = item_categories.columns[item_categories.columns.str.endswith('_category')]
item_categories['simple_category_label'] = from_one_hot_to_label(item_categories, item_simple_categories)

item_categories = item_categories[['item_category_id', 'super_category_label', 'simple_category_label']]

In [12]:
shop_types = shops.columns[shops.columns.str.endswith('_shop_type')]
shops['type_label'] = from_one_hot_to_label(shops, shop_types)

shop_cities = shops.columns[shops.columns.str.endswith('_city')]
shops['city_label'] = from_one_hot_to_label(shops, shop_cities)

shops = shops[['shop_id', 'type_label', 'city_label']]

Adding the shop and item labels to the all_sales table.

In [13]:
all_sales = all_sales.merge(items.merge(item_categories, on='item_category_id', how='left'),
                            on='item_id', how='left').merge(shops, on='shop_id', how='left')

In [14]:
all_sales.info()
all_sales.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11128050 entries, 0 to 11128049
Data columns (total 9 columns):
 #   Column                 Dtype
---  ------                 -----
 0   date_block_num         int8 
 1   shop_id                int64
 2   item_id                int32
 3   item_cnt_day           int32
 4   item_category_id       int32
 5   super_category_label   int8 
 6   simple_category_label  int8 
 7   type_label             int8 
 8   city_label             int8 
dtypes: int32(3), int64(1), int8(5)
memory usage: 350.2 MB


,date_block_num,shop_id,item_id,item_cnt_day,item_category_id,super_category_label,simple_category_label,type_label,city_label
0,1,57,30,31,40,9,31,0,27
1,1,57,31,11,37,9,26,0,27
2,1,57,32,10,40,9,31,0,27
3,1,57,33,3,37,9,26,0,27
4,1,57,35,14,40,9,31,0,27


Little reducing memory and clipping the target feature.

In [15]:
all_sales.shop_id = all_sales.shop_id.astype('int32')
all_sales.item_cnt_day = all_sales.item_cnt_day.clip(0, 20)
all_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11128050 entries, 0 to 11128049
Data columns (total 9 columns):
 #   Column                 Dtype
---  ------                 -----
 0   date_block_num         int8 
 1   shop_id                int32
 2   item_id                int32
 3   item_cnt_day           int32
 4   item_category_id       int32
 5   super_category_label   int8 
 6   simple_category_label  int8 
 7   type_label             int8 
 8   city_label             int8 
dtypes: int32(4), int8(5)
memory usage: 307.8 MB


In [16]:
all_sales.rename(columns={'item_cnt_day': 'item_cnt'}, inplace=True)

It is needed to do experiments to research if the lag features are necessary.

In [17]:
all_sales_without_lags = all_sales.copy()

In [20]:
all_sales_without_lags_and_origin_ids = all_sales_without_lags.drop(columns=['shop_id', 'item_id', 'item_category_id'])
all_sales_without_lags_and_origin_ids.head()

,date_block_num,item_cnt,super_category_label,simple_category_label,type_label,city_label
0,1,20,9,31,0,27
1,1,11,9,26,0,27
2,1,10,9,31,0,27
3,1,3,9,26,0,27
4,1,14,9,31,0,27


Creating lag features.

In [18]:
for i in range(1, 13):
    all_sales = lag_feature(all_sales, i, 'item_cnt', ['date_block_num', 'shop_id', 'item_id'])

KeyboardInterrupt: 

For rows until the January of 2014 the lag feature item_cnt_lag12 is obviously undefined. So these rows are removed.

In [ ]:
all_sales = all_sales[all_sales.date_block_num >= 12]

In [ ]:
all_sales.info()

Creating the train, valid and test datasets.

In [ ]:
X_train = all_sales[all_sales.date_block_num < 33]
X_train, y_train = X_train.drop(columns=['item_cnt']), X_train.item_cnt

X_val = all_sales[all_sales.date_block_num == 33]
X_val, y_val = X_val.drop(columns=['item_cnt']), X_val.item_cnt

X_test = all_sales[all_sales.date_block_num == 34]
X_test, y_test = X_test.drop(columns=['item_cnt']), X_test.item_cnt

Creating the train, valid and test datasets without lag features. So the train dataset also contain the information of the 2013-th year.

In [53]:
X_train_without_lags = all_sales_without_lags[all_sales_without_lags.date_block_num < 33]
X_train_without_lags, y_train_without_lags = X_train_without_lags.drop(columns=['item_cnt']), X_train_without_lags.item_cnt

X_val_without_lags = all_sales_without_lags[all_sales_without_lags.date_block_num == 33]
X_val_without_lags, y_val_without_lags = X_val_without_lags.drop(columns=['item_cnt']), X_val_without_lags.item_cnt

X_test_without_lags = all_sales_without_lags[all_sales_without_lags.date_block_num == 34]
X_test_without_lags, y_test_without_lags = X_test_without_lags.drop(columns=['item_cnt']), X_test_without_lags.item_cnt

Creating the train, valid and test datasets without lag features and shop, item and item_categories ids.

In [23]:
X_train_without_lags_and_origin_ids = all_sales_without_lags_and_origin_ids[all_sales_without_lags_and_origin_ids.date_block_num < 33]
y_train_without_lags_and_origin_ids = X_train_without_lags_and_origin_ids.item_cnt
X_train_without_lags_and_origin_ids = X_train_without_lags_and_origin_ids.drop(columns=['item_cnt'])

X_val_without_lags_and_origin_ids = all_sales_without_lags_and_origin_ids[all_sales_without_lags_and_origin_ids.date_block_num == 33]
y_val_without_lags_and_origin_ids = X_val_without_lags_and_origin_ids.item_cnt
X_val_without_lags_and_origin_ids = X_val_without_lags_and_origin_ids.drop(columns=['item_cnt'])

X_test_without_lags_and_origin_ids = all_sales_without_lags_and_origin_ids[all_sales_without_lags_and_origin_ids.date_block_num == 34]
y_test_without_lags_and_origin_ids = X_test_without_lags_and_origin_ids.item_cnt
X_test_without_lags_and_origin_ids = X_test_without_lags_and_origin_ids.drop(columns=['item_cnt'])

LGBM with lag features. I interrupted the kernel because the evaluation time was 2 hours and 47 minutes.

In [24]:
for max_depth in range(1, 9):
    print('max_depth:', max_depth)
    for learning_rate in [0.001, 0.005, 0.01, 0.05, 0.1]:
        print('learning_rate:', learning_rate)
        lgbm = LGBMRegressor(n_estimators=500, max_depth=max_depth, learning_rate=learning_rate)
        lgbm.fit(X_train, y_train)
        print('train MSE:', MSE(y_train, lgbm.predict(X_train).clip(0, 20)), ' | ', 'val MSE:', MSE(y_val, lgbm.predict(X_val).clip(0, 20)))
        print()
    print('------------------------------------------------------')

max_depth: 1
learning_rate: 0.001
train MSE: 1.095962615314234  |  val MSE: 1.0763562729158567

learning_rate: 0.005
train MSE: 0.9768033011890065  |  val MSE: 1.001453345325884

learning_rate: 0.01
train MSE: 0.9439501488933028  |  val MSE: 0.9813068496617915

learning_rate: 0.05
train MSE: 0.9265822280150141  |  val MSE: 0.9691955580332376

learning_rate: 0.1
train MSE: 0.9253391992397709  |  val MSE: 0.9681577568901751

------------------------------------------------------
max_depth: 2
learning_rate: 0.001
train MSE: 1.0590534720009877  |  val MSE: 1.0504024338798983

learning_rate: 0.005
train MSE: 0.9399787780196266  |  val MSE: 0.9763319047258446

learning_rate: 0.01
train MSE: 0.9250243129120677  |  val MSE: 0.9673339707027268

learning_rate: 0.05
train MSE: 0.9146711521210809  |  val MSE: 0.9580111486193177

learning_rate: 0.1
train MSE: 0.9103928469073523  |  val MSE: 0.9545165736597221

------------------------------------------------------
max_depth: 3
learning_rate: 0.001


KeyboardInterrupt: 

In [30]:
for max_depth in range(1, 9):
    print('max_depth:', max_depth)
    lgbm = LGBMRegressor(max_depth=max_depth)
    lgbm.fit(X_train_without_lags_and_origin_ids,
             y_train_without_lags_and_origin_ids,
             eval_set=[(X_val_without_lags_and_origin_ids, y_val_without_lags_and_origin_ids)],
             eval_metric='l2',
             early_stopping_rounds=20,
             verbose=10)
    print('train MSE:', MSE(y_train_without_lags_and_origin_ids, lgbm.predict(X_train_without_lags_and_origin_ids).clip(0, 20)), ' | ',
          'val MSE:', MSE(y_val_without_lags_and_origin_ids, lgbm.predict(X_val_without_lags_and_origin_ids).clip(0, 20)))
    print('------------------------------------------------------')

max_depth: 1


c:\users\user\appdata\local\programs\python\python39\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[10]	valid_0's l2: 1.2866
[20]	valid_0's l2: 1.28284
[30]	valid_0's l2: 1.28083
[40]	valid_0's l2: 1.27977
[50]	valid_0's l2: 1.27743
[60]	valid_0's l2: 1.27624
[70]	valid_0's l2: 1.27393
[80]	valid_0's l2: 1.27365
[90]	valid_0's l2: 1.27201
[100]	valid_0's l2: 1.27138
train MSE: 1.2029902387074427  |  val MSE: 1.1274861203037285
------------------------------------------------------
max_depth: 2
[10]	valid_0's l2: 1.27042
[20]	valid_0's l2: 1.26581
[30]	valid_0's l2: 1.25987
[40]	valid_0's l2: 1.25597
[50]	valid_0's l2: 1.25432
[60]	valid_0's l2: 1.25013
[70]	valid_0's l2: 1.24816
[80]	valid_0's l2: 1.24352
[90]	valid_0's l2: 1.24011
[100]	valid_0's l2: 1.23595
train MSE: 1.1878138337219746  |  val MSE: 1.1117299421278757
------------------------------------------------------
max_depth: 3
[10]	valid_0's l2: 1.24963
[20]	valid_0's l2: 1.23842
[30]	valid_0's l2: 1.22701
[40]	valid_0's l2: 1.2213
[50]	valid_0's l2: 1.21521
[60]	valid_0's l2: 1.19822
[70]	valid_0's l2: 1.18542
[80]	valid_

Creating lag features.

In [102]:
for max_depth in range(1, 8):
    print('max_depth:', max_depth)
    xgb = XGBRegressor(n_estimators=100, max_depth=max_depth)
    xgb.fit(X_train, y_train)
    print('train MSE:', MSE(y_train, xgb.predict(X_train).clip(0, 20)), ' | ', 'val MSE:', MSE(y_val, xgb.predict(X_val).clip(0, 20)))

max_depth: 1
train MSE: 0.9292286170038948  |  val MSE: 0.9715517056653525
max_depth: 2
train MSE: 0.914220336935386  |  val MSE: 0.9583789865727568
max_depth: 3
train MSE: 0.9003089591874074  |  val MSE: 0.9497599942201541
max_depth: 4


KeyboardInterrupt: 

LGBM without lag features

In [85]:
for max_depth in range(1, 11):
    print('max_depth:', max_depth)
    lgbm = LGBMRegressor(n_estimators=100, max_depth=max_depth)
    lgbm.fit(X_train_without_lags, y_train_without_lags)
    print('MSE:', MSE(y_val_without_lags, lgbm.predict(X_val_without_lags)))

max_depth: 1
MSE: 1.1250655557746896
max_depth: 2
MSE: 1.1062470898531294
max_depth: 3
MSE: 1.0799633329985878
max_depth: 4
MSE: 1.0553583578436607
max_depth: 5
MSE: 1.0378690599988074
max_depth: 6
MSE: 1.0349645188174152
max_depth: 7
MSE: 1.0298975810395792
max_depth: 8
MSE: 1.0299027230596054
max_depth: 9
MSE: 1.0327958746789292
max_depth: 10
MSE: 1.0282905971478133


In [26]:
X_final_train = all_sales[all_sales.date_block_num < 34]
X_final_train, y_final_train = X_final_train.drop(columns=['item_cnt']), X_final_train.item_cnt

In [27]:
best_lgbm = LGBMRegressor(n_estimators=100, max_depth=7)
best_lgbm.fit(X_final_train, y_final_train)

submission = pd.read_csv('data/sample_submission.csv')
submission.item_cnt_month = best_lgbm.predict(X_test)
submission.to_csv('data/my_submission_7.csv', index=None)

SCORE = 1.38356

In [84]:
submission_7 = pd.read_csv('data/my_submission_7.csv')
submission_7.item_cnt_month = submission_7.item_cnt_month.clip(0, 20)
submission_7.to_csv('data/my_submission_8.csv', index=None)

It is just the clipped previous submission.
SCORE = 1.38327

In [87]:
X_final_train_without_lags = all_sales_without_lags[all_sales_without_lags.date_block_num < 34]
X_final_train_without_lags, y_final_train_without_lags = X_final_train_without_lags.drop(columns=['item_cnt']), X_final_train_without_lags.item_cnt

In [90]:
best_lgbm_without_lags = LGBMRegressor(n_estimators=100, max_depth=7)
best_lgbm_without_lags.fit(X_final_train_without_lags, y_final_train_without_lags)

submission = pd.read_csv('data/sample_submission.csv')
submission.item_cnt_month = best_lgbm_without_lags.predict(X_test_without_lags).clip(0, 20)
submission.to_csv('data/my_submission_9.csv', index=None)

In [92]:
all_sales_without_lags.head()

,date_block_num,shop_id,item_id,item_cnt,item_category_id,super_category_label,simple_category_label,type_label,city_label
0,1,57,30,20,40,9,31,0,27
1,1,57,31,11,37,9,26,0,27
2,1,57,32,10,40,9,31,0,27
3,1,57,33,3,37,9,26,0,27
4,1,57,35,14,40,9,31,0,27


In [27]:
best_lgbm = LGBMRegressor(n_estimators=500, max_depth=4, learning_rate=0.001)
best_lgbm.fit(X_final_train, y_final_train)

submission = pd.read_csv('data/sample_submission.csv')
submission.item_cnt_month = best_lgbm.predict(X_test).clip(0, 20)
submission.to_csv('data/my_submission_10.csv', index=None)

SCORE = 1.24015

In [44]:
X_final_train_without_lags_and_origin_ids = all_sales_without_lags_and_origin_ids[all_sales_without_lags_and_origin_ids.date_block_num < 34]
X_final_train_without_lags_and_origin_ids, y_final_train_without_lags_and_origin_ids = X_final_train_without_lags_and_origin_ids.drop(columns=['item_cnt']), X_final_train_without_lags_and_origin_ids.item_cnt

In [45]:
best_lgbm = LGBMRegressor(n_estimators=100, max_depth=8)
best_lgbm.fit(X_final_train_without_lags_and_origin_ids, y_final_train_without_lags_and_origin_ids)

submission = pd.read_csv('data/sample_submission.csv')
submission.item_cnt_month = best_lgbm.predict(X_test_without_lags_and_origin_ids).clip(0, 20)
submission.to_csv('data/my_submission_11.csv', index=None)

SCORE = 1.27978